In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
import xgboost as xgb
from time import time
pd.set_option('display.max_columns', 999)

In [2]:
#traindf = pd.read_csv('/Users/Vishy/Files/AVDatahack/Churn/RawData/train.csv')
traindf = pd.read_csv('/data/train.csv')
print (traindf.shape)
traindf.head()

/usr/local/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (294,299,300,302,307,313,315,365,367,369) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(300000, 377)


,UCIC_ID,NO_OF_Accs,HNW_CATEGORY,vintage,EMAIL_UNSUBSCRIBE,OCCUP_ALL_NEW,city,dependents,zip,FINAL_WORTH_prev1,ENGAGEMENT_TAG_prev1,C_prev1,D_prev1,ATM_C_prev1,ATM_D_prev1,BRANCH_C_prev1,BRANCH_D_prev1,IB_C_prev1,IB_D_prev1,MB_C_prev1,MB_D_prev1,POS_C_prev1,POS_D_prev1,count_C_prev1,count_D_prev1,COUNT_ATM_C_prev1,COUNT_ATM_D_prev1,COUNT_BRANCH_C_prev1,COUNT_BRANCH_D_prev1,COUNT_IB_C_prev1,COUNT_IB_D_prev1,COUNT_MB_C_prev1,COUNT_MB_D_prev1,COUNT_POS_C_prev1,COUNT_POS_D_prev1,custinit_CR_amt_prev1,custinit_DR_amt_prev1,custinit_CR_cnt_prev1,custinit_DR_cnt_prev1,ATM_amt_prev1,ATM_CW_Amt_prev1,ATM_CW_Cnt_prev1,BRN_CW_Amt_prev1,BRN_CW_Cnt_prev1,BRN_CASH_Dep_Amt_prev1,BRN_CASH_Dep_Cnt_prev1,CNR_prev1,BAL_prev1,EOP_prev1,CR_AMB_Prev1,C_prev2,D_prev2,ATM_C_prev2,ATM_D_prev2,BRANCH_C_prev2,BRANCH_D_prev2,IB_C_prev2,IB_D_prev2,MB_C_prev2,MB_D_prev2,POS_C_prev2,POS_D_prev2,count_C_prev2,count_D_prev2,COUNT_ATM_C_prev2,COUNT_ATM_D_prev2,COUNT_BRANCH_C_prev2,COUNT_BRANCH_D_prev2,COUNT_IB_C_prev2,COUNT_IB_D_prev2,COUNT_MB_C_prev2,COUNT_MB_D_prev2,COUNT_POS_C_prev2,COUNT_POS_D_prev2,custinit_CR_amt_prev2,custinit_DR_amt_prev2,custinit_CR_cnt_prev2,custinit_DR_cnt_prev2,ATM_amt_prev2,ATM_CW_Amt_prev2,ATM_CW_Cnt_prev2,BRN_CW_Amt_prev2,BRN_CW_Cnt_prev2,BRN_CASH_Dep_Amt_prev2,BRN_CASH_Dep_Cnt_prev2,CNR_prev2,BAL_prev2,EOP_prev2,CR_AMB_Prev2,C_prev3,D_prev3,ATM_C_prev3,ATM_D_prev3,BRANCH_C_prev3,BRANCH_D_prev3,IB_C_prev3,IB_D_prev3,MB_C_prev3,MB_D_prev3,POS_C_prev3,POS_D_prev3,count_C_prev3,count_D_prev3,COUNT_ATM_C_prev3,COUNT_ATM_D_prev3,COUNT_BRANCH_C_prev3,COUNT_BRANCH_D_prev3,COUNT_IB_C_prev3,COUNT_IB_D_prev3,COUNT_MB_C_prev3,COUNT_MB_D_prev3,COUNT_POS_C_prev3,COUNT_POS_D_prev3,custinit_CR_amt_prev3,custinit_DR_amt_prev3,custinit_CR_cnt_prev3,custinit_DR_cnt_prev3,ATM_amt_prev3,ATM_CW_Amt_prev3,ATM_CW_Cnt_prev3,BRN_CW_Amt_prev3,BRN_CW_Cnt_prev3,BRN_CASH_Dep_Amt_prev3,BRN_CASH_Dep_Cnt_prev3,CNR_prev3,BAL_prev3,EOP_prev3,CR_AMB_Prev3,C_prev4,D_prev4,ATM_C_prev4,ATM_D_prev4,BRANCH_C_prev4,BRANCH_D_prev4,IB_C_prev4,IB_D_prev4,MB_C_prev4,MB_D_prev4,POS_C_prev4,POS_D_prev4,count_C_prev4,count_D_prev4,COUNT_ATM_C_prev4,COUNT_ATM_D_prev4,COUNT_BRANCH_C_prev4,COUNT_BRANCH_D_prev4,COUNT_IB_C_prev4,COUNT_IB_D_prev4,COUNT_MB_C_prev4,COUNT_MB_D_prev4,COUNT_POS_C_prev4,COUNT_POS_D_prev4,custinit_CR_amt_prev4,custinit_DR_amt_prev4,custinit_CR_cnt_prev4,custinit_DR_cnt_prev4,ATM_amt_prev4,ATM_CW_Amt_prev4,ATM_CW_Cnt_prev4,BRN_CW_Amt_prev4,BRN_CW_Cnt_prev4,BRN_CASH_Dep_Amt_prev4,BRN_CASH_Dep_Cnt_prev4,CNR_prev4,BAL_prev4,EOP_prev4,CR_AMB_Prev4,C_prev5,D_prev5,ATM_C_prev5,ATM_D_prev5,BRANCH_C_prev5,BRANCH_D_prev5,IB_C_prev5,IB_D_prev5,MB_C_prev5,MB_D_prev5,POS_C_prev5,POS_D_prev5,count_C_prev5,count_D_prev5,COUNT_ATM_C_prev5,COUNT_ATM_D_prev5,COUNT_BRANCH_C_prev5,COUNT_BRANCH_D_prev5,COUNT_IB_C_prev5,COUNT_IB_D_prev5,COUNT_MB_C_prev5,COUNT_MB_D_prev5,COUNT_POS_C_prev5,COUNT_POS_D_prev5,custinit_CR_amt_prev5,custinit_DR_amt_prev5,custinit_CR_cnt_prev5,custinit_DR_cnt_prev5,ATM_amt_prev5,ATM_CW_Amt_prev5,ATM_CW_Cnt_prev5,BRN_CW_Amt_prev5,BRN_CW_Cnt_prev5,BRN_CASH_Dep_Amt_prev5,BRN_CASH_Dep_Cnt_prev5,CNR_prev5,BAL_prev5,EOP_prev5,CR_AMB_Prev5,C_prev6,D_prev6,ATM_C_prev6,ATM_D_prev6,BRANCH_C_prev6,BRANCH_D_prev6,IB_C_prev6,IB_D_prev6,MB_C_prev6,MB_D_prev6,POS_C_prev6,POS_D_prev6,count_C_prev6,count_D_prev6,COUNT_ATM_C_prev6,COUNT_ATM_D_prev6,COUNT_BRANCH_C_prev6,COUNT_BRANCH_D_prev6,COUNT_IB_C_prev6,COUNT_IB_D_prev6,COUNT_MB_C_prev6,COUNT_MB_D_prev6,COUNT_POS_C_prev6,COUNT_POS_D_prev6,custinit_CR_amt_prev6,custinit_DR_amt_prev6,custinit_CR_cnt_prev6,custinit_DR_cnt_prev6,ATM_amt_prev6,ATM_CW_Amt_prev6,ATM_CW_Cnt_prev6,BRN_CW_Amt_prev6,BRN_CW_Cnt_prev6,BRN_CASH_Dep_Amt_prev6,BRN_CASH_Dep_Cnt_prev6,CNR_prev6,BAL_prev6,EOP_prev6,CR_AMB_Prev6,FRX_PrevQ1,EFT_SELF_TRANSFER_PrevQ1,Billpay_Active_PrevQ1,Billpay_Reg_ason_Prev1,FD_AMOUNT_BOOK_PrevQ1,FD_AMOUNT_BOOK_PrevQ2,NO_OF_FD_BOOK_PrevQ1,NO_OF_FD_BOOK_PrevQ2,NO_OF_RD_BOOK_PrevQ1,NO_OF_RD_BOOK_PrevQ2,RD_AMOUNT_BOOK_Pr

In [3]:
# There are 333 variables in the training set data
nonnumcols = traindf.dtypes[traindf.dtypes == "object"].index
print (len(nonnumcols))
#print (nonnumcols)

38


In [4]:
del traindf['city'], traindf['zip']

In [5]:
# Filling in the missing value
traindf['OCCUP_ALL_NEW'] = traindf['OCCUP_ALL_NEW'].fillna(pd.Series(np.random.choice(['SELF_EMPLOYED','SALARIED','HOUSEWIFE'], p=[0.50,0.30,0.20], size=len(traindf))))
traindf['dependents'] = traindf['dependents'].fillna(pd.Series(np.random.choice([0,2,1], p=[0.85,0.10,0.05], size=len(traindf))))
traindf['FINAL_WORTH_prev1'] = traindf['FINAL_WORTH_prev1'].fillna(pd.Series(np.random.choice(['HIGH','MEDIUM'], p=[0.75,0.25], size=len(traindf))))
traindf['ENGAGEMENT_TAG_prev1'] = traindf['ENGAGEMENT_TAG_prev1'].fillna(pd.Series(np.random.choice(['LOW','HIGH','MEDIUM'], p=[0.50,0.30,0.20], size=len(traindf))))

In [6]:
# Only Y columns
traindf['EMAIL_UNSUBSCRIBE'] = np.where(traindf['EMAIL_UNSUBSCRIBE']=='Y', 1, 0)
traindf['AGRI_TAG_LIVE'] = np.where(traindf['AGRI_TAG_LIVE']=='Y', 1, 0)
traindf['AL_CNC_TAG_LIVE'] = np.where(traindf['AL_CNC_TAG_LIVE']=='Y', 1, 0)
traindf['AL_TAG_LIVE'] = np.where(traindf['AL_TAG_LIVE']=='Y', 1, 0)
traindf['BL_TAG_LIVE'] = np.where(traindf['BL_TAG_LIVE']=='Y', 1, 0)
traindf['CC_TAG_LIVE'] = np.where(traindf['CC_TAG_LIVE']=='Y', 1, 0)
traindf['CE_TAG_LIVE'] = np.where(traindf['CE_TAG_LIVE']=='Y', 1, 0)
traindf['CV_TAG_LIVE'] = np.where(traindf['CV_TAG_LIVE']=='Y', 1, 0)
traindf['DEMAT_TAG_LIVE'] = np.where(traindf['DEMAT_TAG_LIVE']=='Y', 1, 0)
traindf['EDU_TAG_LIVE'] = np.where(traindf['EDU_TAG_LIVE']=='Y', 1, 0)
traindf['GL_TAG_LIVE'] = np.where(traindf['GL_TAG_LIVE']=='Y', 1, 0)
traindf['HL_TAG_LIVE'] = np.where(traindf['HL_TAG_LIVE']=='Y', 1, 0)
traindf['SEC_ACC_TAG_LIVE'] = np.where(traindf['SEC_ACC_TAG_LIVE']=='Y', 1, 0)
traindf['INS_TAG_LIVE'] = np.where(traindf['INS_TAG_LIVE']=='Y', 1, 0)
traindf['LAS_TAG_LIVE'] = np.where(traindf['LAS_TAG_LIVE']=='Y', 1, 0)
traindf['MF_TAG_LIVE'] = np.where(traindf['MF_TAG_LIVE']=='Y', 1, 0)
traindf['OTHER_LOANS_TAG_LIVE'] = np.where(traindf['OTHER_LOANS_TAG_LIVE']=='Y', 1, 0)
traindf['PL_TAG_LIVE'] = np.where(traindf['PL_TAG_LIVE']=='Y', 1, 0)
traindf['RD_TAG_LIVE'] = np.where(traindf['RD_TAG_LIVE']=='Y', 1, 0)
traindf['FD_TAG_LIVE'] = np.where(traindf['FD_TAG_LIVE']=='Y', 1, 0)
traindf['TL_TAG_LIVE'] = np.where(traindf['TL_TAG_LIVE']=='Y', 1, 0)
traindf['TWL_TAG_LIVE'] = np.where(traindf['TWL_TAG_LIVE']=='Y', 1, 0)
traindf['lap_tag_live'] = np.where(traindf['lap_tag_live']=='Y', 1, 0)
traindf['EFT_SELF_TRANSFER_PrevQ1'] = np.where(traindf['EFT_SELF_TRANSFER_PrevQ1']=='Y', 1, 0)

In [7]:
# Conv the Query/Request as String
traindf['Req_Resolved_PrevQ1']=traindf['Req_Resolved_PrevQ1'].astype(str)
traindf['Query_Resolved_PrevQ1']=traindf['Query_Resolved_PrevQ1'].astype(str)
traindf['Complaint_Resolved_PrevQ1']=traindf['Complaint_Resolved_PrevQ1'].astype(str)

In [8]:
# Replace missing value for Qury/Requests
traindf['Req_Resolved_PrevQ1'] = traindf['Req_Resolved_PrevQ1'].fillna('MISS')
traindf['Query_Resolved_PrevQ1'] = traindf['Query_Resolved_PrevQ1'].fillna('MISS')
traindf['Complaint_Resolved_PrevQ1'] = traindf['Complaint_Resolved_PrevQ1'].fillna('MISS')

In [9]:
# Binning the Request/Query into Categories
def func1(row):
    if row['Req_Resolved_PrevQ1']=='0.0':
        val1 = 'NoReq'
    elif (row['Req_Resolved_PrevQ1']=='1.0')|(row['Req_Resolved_PrevQ1']=='2.0')|(row['Req_Resolved_PrevQ1']=='1'):
        val1 = 'MinReq'
    elif (row['Req_Resolved_PrevQ1']=='3.0')|(row['Req_Resolved_PrevQ1']=='4.0')|(row['Req_Resolved_PrevQ1']=='5.0')|(row['Req_Resolved_PrevQ1']=='3')|(row['Req_Resolved_PrevQ1']=='4'):
        val1 = 'MidReq'
    elif row['Req_Resolved_PrevQ1']=='MISS':
        val1 = 'MISSReq'
    else:
        val1 = 'HighReq'
    return val1

def func2(row):
    if row['Query_Resolved_PrevQ1']=='0.0':
        val2 = 'NoQryRes'
    elif (row['Query_Resolved_PrevQ1']=='1.0')|(row['Query_Resolved_PrevQ1']=='2.0')|(row['Query_Resolved_PrevQ1']=='1')|(row['Query_Resolved_PrevQ1']=='2'):
        val2 = 'MinQryRes'
    elif (row['Query_Resolved_PrevQ1']=='3.0')|(row['Query_Resolved_PrevQ1']=='4.0')|(row['Query_Resolved_PrevQ1']=='5.0')|(row['Query_Resolved_PrevQ1']=='3')|(row['Query_Resolved_PrevQ1']=='4')|(row['Query_Resolved_PrevQ1']=='5'):
        val2 = 'MidQryRes'
    elif row['Query_Resolved_PrevQ1']=='MISS':
        val2 = 'MISSRes'
    else:
        val2 = 'HighQryRes'
    return val2

def func3(row):
    if row['Complaint_Resolved_PrevQ1']=='0.0':
        val3 = 'NoComp'
    elif (row['Complaint_Resolved_PrevQ1']=='1.0')|(row['Complaint_Resolved_PrevQ1']=='2.0'):
        val3 = 'MinComp'
    elif (row['Complaint_Resolved_PrevQ1']=='3.0')|(row['Complaint_Resolved_PrevQ1']=='4.0')|(row['Complaint_Resolved_PrevQ1']=='5.0')|(row['Complaint_Resolved_PrevQ1']=='5'):
        val3 = 'MidComp'
    elif row['Complaint_Resolved_PrevQ1']=='MISS':
        val3 = 'MISSComp'
    else:
        val3 = 'HighComp'
    return val3

In [12]:
#traindf['ReqClass'] = traindf.apply(func1, axis = 1).astype(str)
#traindf['QryClass'] = traindf.apply(func2, axis = 1).astype(str)
traindf['CmpClass'] = traindf.apply(func3, axis = 1).astype(str)

del traindf['Req_Resolved_PrevQ1'],traindf['Query_Resolved_PrevQ1'],traindf['Complaint_Resolved_PrevQ1']

In [13]:
print ('Before One hit encoding', traindf.shape)

Before One hit encoding (300000, 375)


In [14]:
traindf = pd.get_dummies(traindf, columns=['Billpay_Active_PrevQ1_N','HNW_CATEGORY','EMAIL_UNSUBSCRIBE','OCCUP_ALL_NEW',
                                           'FINAL_WORTH_prev1','ENGAGEMENT_TAG_prev1','Billpay_Reg_ason_Prev1_N',
                                           'EFT_SELF_TRANSFER_PrevQ1','Charges_cnt_PrevQ1_N','FRX_PrevQ1_N',
                                           'RBI_Class_Audit','gender_bin','ReqClass','QryClass','CmpClass'], drop_first=True)
print ('After One hot encoding',traindf.shape)

After One hot encoding (300000, 396)


In [15]:
#testdf = pd.read_csv('/Users/Vishy/Files/AVDatahack/Churn/RawData/test.csv')
testdf = pd.read_csv('/data/test.csv')
print (testdf.shape)
testdf.head()

/usr/local/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (294,299,300,302,313,315,365,367,369) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(200000, 376)


,UCIC_ID,NO_OF_Accs,HNW_CATEGORY,vintage,EMAIL_UNSUBSCRIBE,OCCUP_ALL_NEW,city,dependents,zip,FINAL_WORTH_prev1,ENGAGEMENT_TAG_prev1,C_prev1,D_prev1,ATM_C_prev1,ATM_D_prev1,BRANCH_C_prev1,BRANCH_D_prev1,IB_C_prev1,IB_D_prev1,MB_C_prev1,MB_D_prev1,POS_C_prev1,POS_D_prev1,count_C_prev1,count_D_prev1,COUNT_ATM_C_prev1,COUNT_ATM_D_prev1,COUNT_BRANCH_C_prev1,COUNT_BRANCH_D_prev1,COUNT_IB_C_prev1,COUNT_IB_D_prev1,COUNT_MB_C_prev1,COUNT_MB_D_prev1,COUNT_POS_C_prev1,COUNT_POS_D_prev1,custinit_CR_amt_prev1,custinit_DR_amt_prev1,custinit_CR_cnt_prev1,custinit_DR_cnt_prev1,ATM_amt_prev1,ATM_CW_Amt_prev1,ATM_CW_Cnt_prev1,BRN_CW_Amt_prev1,BRN_CW_Cnt_prev1,BRN_CASH_Dep_Amt_prev1,BRN_CASH_Dep_Cnt_prev1,CNR_prev1,BAL_prev1,EOP_prev1,CR_AMB_Prev1,C_prev2,D_prev2,ATM_C_prev2,ATM_D_prev2,BRANCH_C_prev2,BRANCH_D_prev2,IB_C_prev2,IB_D_prev2,MB_C_prev2,MB_D_prev2,POS_C_prev2,POS_D_prev2,count_C_prev2,count_D_prev2,COUNT_ATM_C_prev2,COUNT_ATM_D_prev2,COUNT_BRANCH_C_prev2,COUNT_BRANCH_D_prev2,COUNT_IB_C_prev2,COUNT_IB_D_prev2,COUNT_MB_C_prev2,COUNT_MB_D_prev2,COUNT_POS_C_prev2,COUNT_POS_D_prev2,custinit_CR_amt_prev2,custinit_DR_amt_prev2,custinit_CR_cnt_prev2,custinit_DR_cnt_prev2,ATM_amt_prev2,ATM_CW_Amt_prev2,ATM_CW_Cnt_prev2,BRN_CW_Amt_prev2,BRN_CW_Cnt_prev2,BRN_CASH_Dep_Amt_prev2,BRN_CASH_Dep_Cnt_prev2,CNR_prev2,BAL_prev2,EOP_prev2,CR_AMB_Prev2,C_prev3,D_prev3,ATM_C_prev3,ATM_D_prev3,BRANCH_C_prev3,BRANCH_D_prev3,IB_C_prev3,IB_D_prev3,MB_C_prev3,MB_D_prev3,POS_C_prev3,POS_D_prev3,count_C_prev3,count_D_prev3,COUNT_ATM_C_prev3,COUNT_ATM_D_prev3,COUNT_BRANCH_C_prev3,COUNT_BRANCH_D_prev3,COUNT_IB_C_prev3,COUNT_IB_D_prev3,COUNT_MB_C_prev3,COUNT_MB_D_prev3,COUNT_POS_C_prev3,COUNT_POS_D_prev3,custinit_CR_amt_prev3,custinit_DR_amt_prev3,custinit_CR_cnt_prev3,custinit_DR_cnt_prev3,ATM_amt_prev3,ATM_CW_Amt_prev3,ATM_CW_Cnt_prev3,BRN_CW_Amt_prev3,BRN_CW_Cnt_prev3,BRN_CASH_Dep_Amt_prev3,BRN_CASH_Dep_Cnt_prev3,CNR_prev3,BAL_prev3,EOP_prev3,CR_AMB_Prev3,C_prev4,D_prev4,ATM_C_prev4,ATM_D_prev4,BRANCH_C_prev4,BRANCH_D_prev4,IB_C_prev4,IB_D_prev4,MB_C_prev4,MB_D_prev4,POS_C_prev4,POS_D_prev4,count_C_prev4,count_D_prev4,COUNT_ATM_C_prev4,COUNT_ATM_D_prev4,COUNT_BRANCH_C_prev4,COUNT_BRANCH_D_prev4,COUNT_IB_C_prev4,COUNT_IB_D_prev4,COUNT_MB_C_prev4,COUNT_MB_D_prev4,COUNT_POS_C_prev4,COUNT_POS_D_prev4,custinit_CR_amt_prev4,custinit_DR_amt_prev4,custinit_CR_cnt_prev4,custinit_DR_cnt_prev4,ATM_amt_prev4,ATM_CW_Amt_prev4,ATM_CW_Cnt_prev4,BRN_CW_Amt_prev4,BRN_CW_Cnt_prev4,BRN_CASH_Dep_Amt_prev4,BRN_CASH_Dep_Cnt_prev4,CNR_prev4,BAL_prev4,EOP_prev4,CR_AMB_Prev4,C_prev5,D_prev5,ATM_C_prev5,ATM_D_prev5,BRANCH_C_prev5,BRANCH_D_prev5,IB_C_prev5,IB_D_prev5,MB_C_prev5,MB_D_prev5,POS_C_prev5,POS_D_prev5,count_C_prev5,count_D_prev5,COUNT_ATM_C_prev5,COUNT_ATM_D_prev5,COUNT_BRANCH_C_prev5,COUNT_BRANCH_D_prev5,COUNT_IB_C_prev5,COUNT_IB_D_prev5,COUNT_MB_C_prev5,COUNT_MB_D_prev5,COUNT_POS_C_prev5,COUNT_POS_D_prev5,custinit_CR_amt_prev5,custinit_DR_amt_prev5,custinit_CR_cnt_prev5,custinit_DR_cnt_prev5,ATM_amt_prev5,ATM_CW_Amt_prev5,ATM_CW_Cnt_prev5,BRN_CW_Amt_prev5,BRN_CW_Cnt_prev5,BRN_CASH_Dep_Amt_prev5,BRN_CASH_Dep_Cnt_prev5,CNR_prev5,BAL_prev5,EOP_prev5,CR_AMB_Prev5,C_prev6,D_prev6,ATM_C_prev6,ATM_D_prev6,BRANCH_C_prev6,BRANCH_D_prev6,IB_C_prev6,IB_D_prev6,MB_C_prev6,MB_D_prev6,POS_C_prev6,POS_D_prev6,count_C_prev6,count_D_prev6,COUNT_ATM_C_prev6,COUNT_ATM_D_prev6,COUNT_BRANCH_C_prev6,COUNT_BRANCH_D_prev6,COUNT_IB_C_prev6,COUNT_IB_D_prev6,COUNT_MB_C_prev6,COUNT_MB_D_prev6,COUNT_POS_C_prev6,COUNT_POS_D_prev6,custinit_CR_amt_prev6,custinit_DR_amt_prev6,custinit_CR_cnt_prev6,custinit_DR_cnt_prev6,ATM_amt_prev6,ATM_CW_Amt_prev6,ATM_CW_Cnt_prev6,BRN_CW_Amt_prev6,BRN_CW_Cnt_prev6,BRN_CASH_Dep_Amt_prev6,BRN_CASH_Dep_Cnt_prev6,CNR_prev6,BAL_prev6,EOP_prev6,CR_AMB_Prev6,FRX_PrevQ1,EFT_SELF_TRANSFER_PrevQ1,Billpay_Active_PrevQ1,Billpay_Reg_ason_Prev1,FD_AMOUNT_BOOK_PrevQ1,FD_AMOUNT_BOOK_PrevQ2,NO_OF_FD_BOOK_PrevQ1,NO_OF_FD_BOOK_PrevQ2,NO_OF_RD_BOOK_PrevQ1,NO_OF_RD_BOOK_PrevQ2,RD_AMOUNT_BOOK_Pr

In [16]:
# Properties of attributes
print (testdf.LAS_TAG_LIVE.value_counts(normalize=True))
print ('Missing % is',(testdf.LAS_TAG_LIVE.isnull().sum()/len(testdf)))

Series([], Name: LAS_TAG_LIVE, dtype: float64)
Missing % is 1.0


In [17]:
# Filling in the missing value
testdf['OCCUP_ALL_NEW'] = testdf['OCCUP_ALL_NEW'].fillna(pd.Series(np.random.choice(['SELF_EMPLOYED','SALARIED','HOUSEWIFE'], p=[0.50,0.30,0.20], size=len(testdf))))
testdf['dependents'] = testdf['dependents'].fillna(pd.Series(np.random.choice([0,2,1], p=[0.85,0.10,0.05], size=len(testdf))))
testdf['FINAL_WORTH_prev1'] = testdf['FINAL_WORTH_prev1'].fillna(pd.Series(np.random.choice(['HIGH','MEDIUM'], p=[0.75,0.25], size=len(testdf))))
testdf['ENGAGEMENT_TAG_prev1'] = testdf['ENGAGEMENT_TAG_prev1'].fillna(pd.Series(np.random.choice(['LOW','HIGH','MEDIUM'], p=[0.50,0.30,0.20], size=len(testdf))))

In [18]:
# Only Y columns
testdf['EMAIL_UNSUBSCRIBE'] = np.where(testdf['EMAIL_UNSUBSCRIBE']=='Y', 1, 0)
testdf['AGRI_TAG_LIVE'] = np.where(testdf['AGRI_TAG_LIVE']=='Y', 1, 0)
testdf['AL_CNC_TAG_LIVE'] = np.where(testdf['AL_CNC_TAG_LIVE']=='Y', 1, 0)
testdf['AL_TAG_LIVE'] = np.where(testdf['AL_TAG_LIVE']=='Y', 1, 0)
testdf['BL_TAG_LIVE'] = np.where(testdf['BL_TAG_LIVE']=='Y', 1, 0)
testdf['CC_TAG_LIVE'] = np.where(testdf['CC_TAG_LIVE']=='Y', 1, 0)
testdf['CE_TAG_LIVE'] = np.where(testdf['CE_TAG_LIVE']=='Y', 1, 0)
testdf['CV_TAG_LIVE'] = np.where(testdf['CV_TAG_LIVE']=='Y', 1, 0)
testdf['DEMAT_TAG_LIVE'] = np.where(testdf['DEMAT_TAG_LIVE']=='Y', 1, 0)
testdf['EDU_TAG_LIVE'] = np.where(testdf['EDU_TAG_LIVE']=='Y', 1, 0)
testdf['GL_TAG_LIVE'] = np.where(testdf['GL_TAG_LIVE']=='Y', 1, 0)
testdf['HL_TAG_LIVE'] = np.where(testdf['HL_TAG_LIVE']=='Y', 1, 0)
testdf['SEC_ACC_TAG_LIVE'] = np.where(testdf['SEC_ACC_TAG_LIVE']=='Y', 1, 0)
testdf['INS_TAG_LIVE'] = np.where(testdf['INS_TAG_LIVE']=='Y', 1, 0)
testdf['LAS_TAG_LIVE'] = testdf['LAS_TAG_LIVE'].fillna(0)
testdf['MF_TAG_LIVE'] = np.where(testdf['MF_TAG_LIVE']=='Y', 1, 0)
testdf['OTHER_LOANS_TAG_LIVE'] = np.where(testdf['OTHER_LOANS_TAG_LIVE']=='Y', 1, 0)
testdf['PL_TAG_LIVE'] = np.where(testdf['PL_TAG_LIVE']=='Y', 1, 0)
testdf['RD_TAG_LIVE'] = np.where(testdf['RD_TAG_LIVE']=='Y', 1, 0)
testdf['FD_TAG_LIVE'] = np.where(testdf['FD_TAG_LIVE']=='Y', 1, 0)
testdf['TL_TAG_LIVE'] = np.where(testdf['TL_TAG_LIVE']=='Y', 1, 0)
testdf['TWL_TAG_LIVE'] = np.where(testdf['TWL_TAG_LIVE']=='Y', 1, 0)
testdf['lap_tag_live'] = np.where(testdf['lap_tag_live']=='Y', 1, 0)
testdf['EFT_SELF_TRANSFER_PrevQ1'] = np.where(testdf['EFT_SELF_TRANSFER_PrevQ1']=='Y', 1, 0)

In [19]:
del testdf['city'], testdf['zip']

In [20]:
# Conv the Query/Request as String
testdf['Req_Resolved_PrevQ1']=testdf['Req_Resolved_PrevQ1'].astype(str)
testdf['Query_Resolved_PrevQ1']=testdf['Query_Resolved_PrevQ1'].astype(str)
testdf['Complaint_Resolved_PrevQ1']=testdf['Complaint_Resolved_PrevQ1'].astype(str)

In [21]:
# Replace missing value for Qury/Requests
testdf['Req_Resolved_PrevQ1'] = testdf['Req_Resolved_PrevQ1'].fillna('MISS')
testdf['Query_Resolved_PrevQ1'] = testdf['Query_Resolved_PrevQ1'].fillna('MISS')
testdf['Complaint_Resolved_PrevQ1'] = testdf['Complaint_Resolved_PrevQ1'].fillna('MISS')

In [24]:
#testdf['ReqClass'] = testdf.apply(func1, axis = 1).astype(str)
#testdf['QryClass'] = testdf.apply(func2, axis = 1).astype(str)
testdf['CmpClass'] = testdf.apply(func3, axis = 1).astype(str)

del testdf['Req_Resolved_PrevQ1'],testdf['Query_Resolved_PrevQ1'],testdf['Complaint_Resolved_PrevQ1']

In [25]:
print ('Before One hit encoding', testdf.shape)

Before One hit encoding (200000, 374)


In [26]:
testdf = pd.get_dummies(testdf, columns=['Billpay_Active_PrevQ1_N','HNW_CATEGORY','EMAIL_UNSUBSCRIBE','OCCUP_ALL_NEW',
                                           'FINAL_WORTH_prev1','ENGAGEMENT_TAG_prev1','Billpay_Reg_ason_Prev1_N',
                                           'EFT_SELF_TRANSFER_PrevQ1','Charges_cnt_PrevQ1_N','FRX_PrevQ1_N',
                                           'RBI_Class_Audit','gender_bin','ReqClass','QryClass','CmpClass'], drop_first=True)
print ('After One hot encoding',testdf.shape)

After One hot encoding (200000, 395)


In [27]:
print (traindf.shape, testdf.shape)

(300000, 396) (200000, 395)


In [25]:
listA = list(traindf.columns.values)
listB = list(testdf.columns.values)

#print (len(listA), len(listB))

#for item in listB:
#    if item not in listA:
#        print (item)

In [28]:
allcols = list(traindf.columns.values)
print (len(allcols))

396


In [29]:
df = pd.DataFrame({'col':allcols})

In [30]:
testdf['Responders']=0

In [31]:
traindf = traindf[allcols]
testdf = testdf[allcols]
del testdf['Responders']

In [32]:
def xgb_lift(preds, dtrain):
    y = dtrain.get_label()
    y_pred = preds
    df = pd.DataFrame({'proba':y_pred, 'y':y})
    # 0.2 signifies the top 2 decile
    lift_score = df.sort_values('proba', ascending=False).iloc[:int(len(df)*0.2), 1].sum() / df['y'].sum()
    return [('lift', lift_score)]

In [33]:
target_train = traindf['Responders'].values
id_test = testdf['UCIC_ID'].values

train = np.array(traindf.drop(['Responders','UCIC_ID'], axis = 1))
test = np.array(testdf.drop(['UCIC_ID'], axis = 1))

xgb_preds = []

In [35]:
K = 2
kf = KFold(n_splits = K, random_state = 42, shuffle = True)

In [36]:
start = time()

for train_index, test_index in kf.split(train):
    train_X, valid_X = train[train_index], train[test_index]
    train_y, valid_y = target_train[train_index], target_train[test_index]
    xgb_params = {'eta': 0.02, 'max_depth': 4, 'subsample': 0.5, 'colsample_bytree': 0.9, 
                  'objective': 'binary:logistic', 'eval_metric': 'auc', 'seed': 99, 'silent': True}

    d_train = xgb.DMatrix(train_X, train_y)
    d_valid = xgb.DMatrix(valid_X, valid_y)
    d_test = xgb.DMatrix(test)
    
    watchlist = [(d_train, 'train'), (d_valid, 'valid')]
    model = xgb.train(xgb_params, d_train, 5000,  watchlist, maximize=True, feval=xgb_lift,
                      verbose_eval=100, early_stopping_rounds=50)
                        
    xgb_pred = model.predict(d_test)
    xgb_preds.append(list(xgb_pred))

end = time()
print ('Total time for training is',end-start)
# 9192.85 for k=4
# 2643.70 for k=2 
# 1019.95

[0]	train-auc:0.805811	valid-auc:0.805134	train-lift:0.604413	valid-lift:0.600688
Multiple eval metrics have been passed: 'valid-lift' will be used for early stopping.

Will train until valid-lift hasn't improved in 50 rounds.
[100]	train-auc:0.84487	valid-auc:0.843025	train-lift:0.642238	valid-lift:0.63784
[200]	train-auc:0.856754	valid-auc:0.853851	train-lift:0.659079	valid-lift:0.654193
[300]	train-auc:0.864443	valid-auc:0.860315	train-lift:0.668564	valid-lift:0.662272
[400]	train-auc:0.868668	valid-auc:0.863161	train-lift:0.675532	valid-lift:0.667414
[500]	train-auc:0.871947	valid-auc:0.864973	train-lift:0.681804	valid-lift:0.671164
Stopping. Best iteration:
[540]	train-auc:0.872978	valid-auc:0.865418	train-lift:0.683353	valid-lift:0.671976

[0]	train-auc:0.801092	valid-auc:0.797104	train-lift:0.607415	valid-lift:0.602633
Multiple eval metrics have been passed: 'valid-lift' will be used for early stopping.

Will train until valid-lift hasn't improved in 50 rounds.
[100]	train-auc:0

In [37]:
preds=[]
for i in range(len(xgb_preds[0])):
    sum=0
    for j in range(K):
        sum+=xgb_preds[j][i]
    preds.append(sum / K)

out = pd.DataFrame({'UCIC_ID': id_test, 'Responders': preds})

In [38]:
out.to_csv("pred4.csv", index=False)

In [2]:
#pred1 = 0.68245
#pred2 = 0.68303
pred1=pd.read_csv('/Users/Vishy/Files/AVDatahack/Churn/Preds/pred1.csv')
pred2=pd.read_csv('/Users/Vishy/Files/AVDatahack/Churn/Preds/pred2.csv')
print (pred1.shape, pred2.shape)

(200000, 2) (200000, 2)


In [6]:
pred3 = pd.merge(pred1,pred2, how='inner', left_on='UCIC_ID', right_on='UCIC_ID')

In [7]:
pred3.head()

,Responders_x,UCIC_ID,Responders_y
0,0.291762,337734,0.298781
1,0.025897,488166,0.025314
2,0.038351,410785,0.038097
3,0.177845,389145,0.150706
4,0.207477,221090,0.195784


In [8]:
pred3['Responders'] = (pred3['Responders_x']+pred3['Responders_y'])/2

In [9]:
del pred3['Responders_x'],pred3['Responders_y']

In [10]:
pred3 = pred3[['Responders','UCIC_ID']]

In [11]:
pred3.head()

,Responders,UCIC_ID
0,0.295271,337734
1,0.025606,488166
2,0.038224,410785
3,0.164275,389145
4,0.201630,221090


In [13]:
pred3.to_csv("pred3.csv", index=False)